In [1]:
import urllib2
from bs4 import BeautifulSoup

import dateutil.parser
import re
import pandas as pd
import numpy as np
import pprint as pp


In [2]:
url = "http://realestate.nytimes.com/community/manhattanMarketData.aspx"

page = urllib2.urlopen(url)
soup = BeautifulSoup(page, "html.parser")

In [3]:
print type(soup)

<class 'bs4.BeautifulSoup'>


In [4]:
# print soup.find(class_="table").find_all('tbody')

soup.find('a')['href']

u'http://www.nytimes.com/realestate/'

In [5]:
soup.find_all('table')[1]

<table border="0" cellpadding="0" cellspacing="0" width="100%">\n<tr>\n<td bgcolor="#dddddd" colspan="11"><img height="1" src="/images/spacer.gif" width="1"/></td>\n</tr>\n<tr>\n<td bgcolor="#FFFFFF" colspan="11"><img height="1" src="/images/spacer.gif" width="1"/></td>\n</tr>\n<tr bgcolor="#f5f5f5" height="20">\n<td><div class="p3"><b>Zip Code</b></div></td>\n<td bgcolor="#dddddd" width="1"><img height="1" src="/images/spacer.gif" width="1"/></td>\n<td><div class="p3"><b>Neighborhood</b></div></td>\n<td bgcolor="#dddddd" width="1"><img height="1" src="/images/spacer.gif" width="1"/></td>\n<td align="center" nowrap=""><div class="p3"><b><a href="/community/ManhattanMarketData_Average.aspx?intflag=1">Avg. Sales Price</a></b></div></td>\n<td bgcolor="#dddddd" width="1"><img height="1" src="/images/spacer.gif" width="1"/></td>\n<td align="center" nowrap=""><div class="p3"><b><a href="/community/ManhattanMarketData_Average.aspx?intflag=2">Median Sales Price</a></b></div></td>\n<td bgcolor=

In [6]:
# extract soup for urlname
def url_to_soup(urlname):
    try:
        page = urllib2.urlopen(urlname)
        soup = BeautifulSoup(page, 'html.parser')
    except:
        print urlname
        soup = 'N/A'
        print 'soup', soup
    return soup

In [15]:
def convert_raw_row_to_clean_data(raw_data, skip_row=[], cut_off_rows=0):
    clean_row_data = []
    headers = []
    no_of_rows = len(raw_data)
    row_cnt = 0
    for i, row in enumerate(raw_data):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if len(cols) > 1: # skip the lines in table
            if row_cnt in skip_row or row_cnt >= (no_of_rows - cut_off_rows):
                continue
            elif row_cnt == 0: # can be header row or skipped by user
                headers = [ele.replace(' ', '_').replace('.','').replace('/', '_') 
                           for ele in cols if ele] # get rid of empty values
            else:
                clean_row_data.append([ele for ele in cols if ele]) # get rid of empty values
            row_cnt += 1
    if len(set(headers)) != len(headers):
        headers = []
            
    return headers, clean_row_data

In [16]:
def make_df(row_data, headers):
    try:
        df = pd.DataFrame(row_data, columns=headers)
    except:
        df = pd.DataFrame(row_data)
    
    return df

In [17]:
# extract and clean data from box office mojo seasonal URL
def extract_table_data(urlname, table_number, skip_row_lst=[], cut_off_rows=0):
    soup = url_to_soup(urlname)
    table = soup.find('table', attrs={'class':'lineItemsTable'})     # finds tables in html

    # extracts raw data from url page
    raw_row_data = soup.find_all('table')[table_number].find_all('tr')
#     print raw_row_data
#     return
    headers, clean_row_data = convert_raw_row_to_clean_data(raw_row_data, 
                                                            skip_row_lst, 
                                                            cut_off_rows)
#     print headers
    df = make_df(clean_row_data, headers)

    return df

In [18]:
manh_df = extract_table_data(url,1)

In [19]:
manh_df.dtypes

Zip_Code              object
Neighborhood          object
Avg_Sales_Price       object
Median_Sales_Price    object
Avg_Price_SqFt        object
Avg_SqFt              object
dtype: object

In [20]:
manh_df.loc[0]

Zip_Code                               10001
Neighborhood          Chelsea / West Midtown
Avg_Sales_Price                   $2,015,154
Median_Sales_Price                $1,560,000
Avg_Price_SqFt                        $1,369
Avg_SqFt                               1,472
Name: 0, dtype: object

In [21]:
def convert_value_to_num(value):
    return float(value.strip('$').replace(',',''))

In [22]:
def convert_cols_to_num(df, col_list):
    for col in col_list:
        df[col] = df[col].apply(lambda x: convert_value_to_num(x))
    return df

In [23]:
change_cols = manh_df.columns[2:]

In [24]:
manh_df = convert_cols_to_num(manh_df, change_cols)

In [25]:
# manh_df.set_index('Zip_Code', inplace=True)

In [26]:
manh_df.to_json(path_or_buf='manh_pv_2015_Q2.json',orient='index')

In [27]:
pv_list = manh_df.to_dict('records')

In [28]:
pv_dict = {}
for i in pv_list:
    pv_dict[i['Zip_Code']] = i

pv_dict

{u'10001': {u'Avg_Price_SqFt': 1369.0,
  u'Avg_Sales_Price': 2015154.0,
  u'Avg_SqFt': 1472.0,
  u'Median_Sales_Price': 1560000.0,
  u'Neighborhood': u'Chelsea / West Midtown',
  u'Zip_Code': u'10001'},
 u'10002': {u'Avg_Price_SqFt': 1078.0,
  u'Avg_Sales_Price': 912769.0,
  u'Avg_SqFt': 847.0,
  u'Median_Sales_Price': 750000.0,
  u'Neighborhood': u'Lower East Side',
  u'Zip_Code': u'10002'},
 u'10003': {u'Avg_Price_SqFt': 1343.0,
  u'Avg_Sales_Price': 1838623.0,
  u'Avg_SqFt': 1369.0,
  u'Median_Sales_Price': 1250000.0,
  u'Neighborhood': u'Greenwich Village/Union Square/Gramercy/Flatiron',
  u'Zip_Code': u'10003'},
 u'10004': {u'Avg_Price_SqFt': 1257.0,
  u'Avg_Sales_Price': 1262690.0,
  u'Avg_SqFt': 1005.0,
  u'Median_Sales_Price': 1173000.0,
  u'Neighborhood': u'Battery Park City',
  u'Zip_Code': u'10004'},
 u'10005': {u'Avg_Price_SqFt': 1230.0,
  u'Avg_Sales_Price': 1260806.0,
  u'Avg_SqFt': 1025.0,
  u'Median_Sales_Price': 1050000.0,
  u'Neighborhood': u'Financial District',
  u'

In [30]:

# with open("manh_pv_2015_Q2.json") as json_file:
#     pv_data = json.load(json_file)
#     print(pv_data)

In [31]:
import json

with open("response.json") as json_file:
    json_data = json.load(json_file)
    print(json_data)

{u'type': u'FeatureCollection', u'features': [{u'geometry': {u'type': u'Polygon', u'coordinates': [[[-73.97689056396484, 40.79423522949219], [-73.97417449951172, 40.79655838012695], [-73.97027587890625, 40.80236053466797], [-73.96651458740234, 40.8082275390625], [-73.96589660644531, 40.808807373046875], [-73.96390533447266, 40.80800247192383], [-73.96485137939453, 40.806705474853516], [-73.96200561523438, 40.80550765991211], [-73.95982360839844, 40.80543518066406], [-73.95709228515625, 40.810081481933594], [-73.95780944824219, 40.81126403808594], [-73.95496368408203, 40.810062408447266], [-73.95680236816406, 40.80754470825195], [-73.95964813232422, 40.80115509033203], [-73.95819854736328, 40.80055236816406], [-73.96003723144531, 40.798038482666016], [-73.96701049804688, 40.788475036621094], [-73.97708892822266, 40.792724609375], [-73.97689056396484, 40.79423522949219]]]}, u'type': u'Feature', u'properties': {u'county': u'New york', u'city': u'New york', u'state': u'NY', u'zipCode': u'1

In [32]:
for feature in json_data['features']:
    print type(feature['properties'])
    print feature['properties'].keys()
    print feature['properties']['zipCode']
    print feature['properties']['county']
    print feature['properties']['city']
    print feature['properties']['state']
    break

<type 'dict'>
[u'county', u'city', u'state', u'zipCode']
10025
New york
New york
NY


In [33]:
def extract_zip_info(geojson_dict):
    zip_dict = {}
    for feature in geojson_dict['features']:
        zipCode = feature['properties']['zipCode']
#         nta_name = feature['properties']['NTAName']
        if feature['geometry']['type'] == 'Polygon':
            polygon_coor = feature['geometry']['coordinates'][0]
            zip_dict[zipCode] = {'NTACode': zipCode, 'polygon': polygon_coor}
        else:
            polygon_coor = feature['geometry']['coordinates']
            zip_dict[zipCode] = {'NTACode': zipCode, 'polygon': polygon_coor}
#         pprint(nta_dict)
#         break
    return zip_dict

In [34]:
manh_dict = extract_zip_info(json_data)

In [35]:
def add_prop_values_to_json(json_dict, prop_val_dict):
    for i, zip_cd in enumerate(json_dict['features']):
#         print zip_cd['properties']['zipCode']
        zipCode = zip_cd['properties']['zipCode']
        try:
            zip_cd['properties']['PropValues'] = prop_val_dict[zipCode]
#             print "succus for %s" % zipCode
        except:
            print "Failed to find PROPERTY VALUES for %s" % zipCode

In [36]:
add_prop_values_to_json(json_data, pv_dict)

Failed to find PROPERTY VALUES for 10171
Failed to find PROPERTY VALUES for 10152
Failed to find PROPERTY VALUES for 10279
Failed to find PROPERTY VALUES for 10112
Failed to find PROPERTY VALUES for 10167
Failed to find PROPERTY VALUES for 10119
Failed to find PROPERTY VALUES for 10110
Failed to find PROPERTY VALUES for 10162
Failed to find PROPERTY VALUES for 10169
Failed to find PROPERTY VALUES for 10153
Failed to find PROPERTY VALUES for 10154
Failed to find PROPERTY VALUES for 10173
Failed to find PROPERTY VALUES for 10199
Failed to find PROPERTY VALUES for 10165
Failed to find PROPERTY VALUES for 10282
Failed to find PROPERTY VALUES for 10168
Failed to find PROPERTY VALUES for 10278
Failed to find PROPERTY VALUES for 10020
Failed to find PROPERTY VALUES for 10044
Failed to find PROPERTY VALUES for 10170
Failed to find PROPERTY VALUES for 10172
Failed to find PROPERTY VALUES for 10174
Failed to find PROPERTY VALUES for 10271
Failed to find PROPERTY VALUES for 10115


In [37]:
json_data

{u'features': [{u'geometry': {u'coordinates': [[[-73.97689056396484,
       40.79423522949219],
      [-73.97417449951172, 40.79655838012695],
      [-73.97027587890625, 40.80236053466797],
      [-73.96651458740234, 40.8082275390625],
      [-73.96589660644531, 40.808807373046875],
      [-73.96390533447266, 40.80800247192383],
      [-73.96485137939453, 40.806705474853516],
      [-73.96200561523438, 40.80550765991211],
      [-73.95982360839844, 40.80543518066406],
      [-73.95709228515625, 40.810081481933594],
      [-73.95780944824219, 40.81126403808594],
      [-73.95496368408203, 40.810062408447266],
      [-73.95680236816406, 40.80754470825195],
      [-73.95964813232422, 40.80115509033203],
      [-73.95819854736328, 40.80055236816406],
      [-73.96003723144531, 40.798038482666016],
      [-73.96701049804688, 40.788475036621094],
      [-73.97708892822266, 40.792724609375],
      [-73.97689056396484, 40.79423522949219]]],
    u'type': u'Polygon'},
   u'properties': {'PropVal

In [38]:
# manh_df.to_json(path_or_buf='manh_pv_2015_Q2.json',orient='index')


with open('pv_by_zip.json', 'w') as outfile:
     json.dump(json_data, outfile, sort_keys = True, indent = 4,
ensure_ascii=False)